In [1]:
# Settings
SPLIT = -1
NUM_OF_CLASSES = 35
IMAGE_SIZE_REDUCE_FACTOR = 3
BATCH_SIZE = 8

In [2]:
# Set up processing
from pickle import load

import numpy as np
from multiprocessing import Pool

def pre_process_images(img):
#     res = (img[0].astype(np.float16)/255, img[1][:, :, ontology])
    return (img[0].astype(np.float16)/255, img[1][:, :, ontology])
        

def split_x_y(data):
    x_data = np.array([s[0] for s in data]).reshape((len(data),) + data[0][0].shape)
    y_data = np.array([s[1] for s in data]).reshape((len(data),) + data[0][1].shape)    
    return x_data, y_data

files = "Processed Data/images_"

In [3]:
# Load Ontology
import pandas as pd

ontology = pd.read_csv("Rellis-3D/ontology.csv")[["class_name", "output_value", "display_color"]].values.tolist()
colors = {v[0]: v[2] for v in ontology}
ontology = {v[0]: v[1] for v in ontology}

# Remove extra classes
del ontology["void"]
# del ontology["dirt"]
del ontology["uphill"]
del ontology["downhill"]

# Process colors
colors = {c: (int(colors[c][1:3], 16), int(colors[c][3:5], 16), int(colors[c][5:7], 16)) for c in ontology.keys()}

# Convert ontology to color map
ontology = list(ontology.values())
print(ontology)
ontology = [i in ontology for i in range(NUM_OF_CLASSES)]
    
NUM_OF_CLASSES = ontology.count(True)

[1, 3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 17, 18, 19, 23, 27, 31, 33, 34]


In [4]:
# Disable all GPUS
import tensorflow as tf

tf.config.set_visible_devices([], 'GPU')
visible_devices = tf.config.get_visible_devices()
for device in visible_devices:
    assert device.device_type != 'GPU'

2023-12-13 22:25:17.632290: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-13 22:25:17.675382: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-13 22:25:18.295054: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [5]:
# This class streams data to the model
# https://stackoverflow.com/a/71592809
from tensorflow.keras.utils import Sequence

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

In [6]:
# Create downsizer
from tensorflow.keras import layers, models

model_in = layers.Input(shape=(1200, 1920, 3))
x = layers.AveragePooling2D(IMAGE_SIZE_REDUCE_FACTOR)(model_in)

x_model = models.Model(model_in, x)
x_model.compile('adam', 'mean_squared_error')

model_in = layers.Input(shape=(1200, 1920, NUM_OF_CLASSES))
x = layers.AveragePooling2D(IMAGE_SIZE_REDUCE_FACTOR)(model_in)

y_model = models.Model(model_in, x)
y_model.compile('adam', 'mean_squared_error')

In [7]:
# Preprocess Train

train = load(open(files + "train_%d.pickle" % SPLIT, "rb"))

# Normalize images
print("Processing images")
with Pool(64) as p:
    train = p.map(pre_process_images, train)
print("Processing done")

print("Spliting images")
x_train, y_train = split_x_y(train)
del train

x_train = DataGenerator(x_train, x_train, BATCH_SIZE)
y_train = DataGenerator(y_train, y_train, BATCH_SIZE)

FileNotFoundError: [Errno 2] No such file or directory: 'Processed Data/images_train_-1.pickle'

In [ ]:
# Downsize and save train
from pickle import dump

if IMAGE_SIZE_REDUCE_FACTOR > 1:
    x_train = x_model.predict(x_train)
    y_train = y_model.predict(y_train)

    np.savez("Processed Data/images_%s_%d_downsized_%d.npz" % ("train", SPLIT, IMAGE_SIZE_REDUCE_FACTOR), x=x_train, y=y_train)
    del x_train, y_train
else:
    np.savez("Processed Data/images_%s_%d_downsized_%d.npz" % ("train", SPLIT, IMAGE_SIZE_REDUCE_FACTOR), x=x_train.x, y=y_train.y)

In [ ]:
# Preprocess Val

val = load(open(files + "val_%d.pickle" % SPLIT, "rb"))

# Normalize images
print("Processing images")
with Pool(64) as p:
    val = p.map(pre_process_images, val)
print("Processing done")

print("Spliting images")
x_val, y_val = split_x_y(val)
del val

x_val = DataGenerator(x_val, x_val, BATCH_SIZE)
y_val = DataGenerator(y_val, y_val, BATCH_SIZE)

In [ ]:
# Downsize and save val

if IMAGE_SIZE_REDUCE_FACTOR > 1:
    x_val = x_model.predict(x_val)
    y_val = y_model.predict(y_val)

    np.savez("Processed Data/images_%s_%d_downsized_%d.npz" % ("val", SPLIT, IMAGE_SIZE_REDUCE_FACTOR), x=x_val, y=y_val)
    del x_val, y_val
else:
    np.savez("Processed Data/images_%s_%d_downsized_%d.npz" % ("val", SPLIT, IMAGE_SIZE_REDUCE_FACTOR), x=x_val.x, y=y_val.y)

In [ ]:
# Preprocess Test

test = load(open(files + "test_%d.pickle" % SPLIT, "rb"))

# Normalize images
print("Processing images")
with Pool(64) as p:
    test = p.map(pre_process_images, test)
print("Processing done")

print("Spliting images")
x_test, y_test = split_x_y(test)
del test

x_test = DataGenerator(x_test, x_test, BATCH_SIZE)
y_test = DataGenerator(y_test, y_test, BATCH_SIZE)

In [ ]:
# Downsize and save val

if IMAGE_SIZE_REDUCE_FACTOR > 1:
    x_test = x_model.predict(x_test)
    y_test = y_model.predict(y_test)

    np.savez("Processed Data/images_%s_%d_downsized_%d.npz" % ("test", SPLIT, IMAGE_SIZE_REDUCE_FACTOR), x=x_test, y=y_test)
    del x_test, y_test
else:
    np.savez("Processed Data/images_%s_%d_downsized_%d.npz" % ("test", SPLIT, IMAGE_SIZE_REDUCE_FACTOR), x=np.array(x_test.x), y=np.array(y_test.y))